# French Motor Claims Datasets

This analysis uses the same features as in the book "Computational Actuarial Science with R" by Arthur Charpentier.

Charpentier Arthur and Katrien Antonio. 2015. Computational Actuarial Science with R. Boca Raton Florida: CRC Press.

In [1]:
import pandas as pd
import tarfile
import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_poisson_deviance
from sklearn.preprocessing import OrdinalEncoder,OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from stabletrees import BaseLineTree,AbuTreeI,NaiveUpdate,TreeReevaluation,StabilityRegularization, BootstrapUpdate
SEED = 0
plt.rcParams["figure.figsize"] = (20,12)

In [5]:
path = "..\data\poisson"
for file in os.listdir(path):
    if file.endswith('.csv'):
        name = file.split(".")[0]
        print(name)
        tar = tarfile.open(path+"//"+name+".tar.gz", "w:gz")
        tar.add(path+"//"+file)
        tar.close()

AllstateClaim
freMTPL2freq
freMTPLfreq


In [2]:
with tarfile.open("..\data\poisson\\freMTPLfreq.tar.gz", "r:*") as tar:
    csv_path = tar.getnames()[0]
    df = pd.read_csv(tar.extractfile(csv_path), header=0)

In [3]:
df.head()

,PolicyID,ClaimNb,Exposure,Power,CarAge,DriverAge,Brand,Gas,Region,Density
0,1,0,0.09,g,0,46,Japanese (except Nissan) or Korean,Diesel,Aquitaine,76
1,2,0,0.84,g,0,46,Japanese (except Nissan) or Korean,Diesel,Aquitaine,76
2,3,0,0.52,f,2,38,Japanese (except Nissan) or Korean,Regular,Nord-Pas-de-Calais,3003
3,4,0,0.45,f,2,38,Japanese (except Nissan) or Korean,Regular,Nord-Pas-de-Calais,3003
4,5,0,0.15,g,0,41,Japanese (except Nissan) or Korean,Diesel,Pays-de-la-Loire,60


In [4]:
df["y"] =df.ClaimNb * df.Exposure #annulized claim frequency (see p.480 in computational actuarial science with R)

ClaimNb = df.ClaimNb.to_numpy()
Exposure = df.Exposure.to_numpy()
df.drop(["PolicyID","Exposure","ClaimNb"],axis=1,inplace=True)


## Density
The density of inhabitants (number of inhabitants per km2) in the city the driver of the car lives in.

In [5]:
df.Density = pd.cut(df.Density, include_lowest=True, bins=[0,40,200,500,4500,np.inf])

## DrivAge
The driver age, in years (in France, people can drive a car at 18).

In [6]:
df.DriverAge  = pd.cut(df.DriverAge , bins=[17,22,26,42,74,np.inf])

## VehAge 
The vehicle age, in years.

In [7]:
df.CarAge  = pd.cut(df.CarAge, include_lowest=True , bins=[0,15,np.inf])

## Brand


In [8]:
df["brandF"] = np.where(df.Brand=="Japanese (except Nissan) or Korean","F","other")

## Power

In [9]:
df["Power"] = ["DEF" if p in ["d","e","f"] else "other" if p in ["d","e","f"] else "GH" for p in df.Power ]

In [10]:
df.insert(len(df.columns)-1, 'y', df.pop('y'))

In [31]:
tree_preprocessor = ColumnTransformer(
    [
        (
            "categorical",
            OrdinalEncoder(),
            ["CarAge", "DriverAge", "Gas", "Density"],
        ),
        (
            "onehot_categorical",
            OneHotEncoder(),
            ["Power", "brandF"],
        ),
        ("numeric", "passthrough", ["y"]),
    ],
    remainder="drop",
)

In [32]:
df_prep = tree_preprocessor.fit_transform(df)
X = df_prep[:,:-1]
y = df_prep[:,-1]


In [24]:
t = BaseLineTree(criterion="poisson",adaptive_complexity=True).fit(X,y)
mask = ClaimNb!=0
mean_poisson_deviance(ClaimNb,t.predict(X)*Exposure)


0.27918505332115984

In [36]:
from sklearn.linear_model import PoissonRegressor
lm = PoissonRegressor(alpha=0.001).fit(X,y)
mask = ClaimNb!=0
mean_poisson_deviance(ClaimNb,lm.predict(X)*Exposure)



0.27981742095821643

In [14]:
from sklearn.model_selection import train_test_split,GridSearchCV,RepeatedKFold
EPSILON = 1.1

def S1(pred1, pred2):
    return np.std(np.log((pred2+EPSILON)/(pred1+EPSILON)))

def S2(pred1, pred2):
    return np.mean((pred1- pred2)**2)

In [17]:
models = {  
            "baseline": BaseLineTree(),
            "NU": NaiveUpdate(),
            "TR":TreeReevaluation(delta=0.1),
            "SR":StabilityRegularization(),
            #"BT": BootstrapUpdate(),
            "ABU":AbuTreeI()
            }
stability_all = {name:[] for name in models.keys()}
standard_stability_all= {name:[] for name in models.keys()}
mse_all= {name:[] for name in models.keys()}

stability = {name:[] for name in models.keys()}
standard_stability = {name:[] for name in models.keys()}
mse = {name:[] for name in models.keys()}
train_stability = {name:[] for name in models.keys()}
train_standard_stability = {name:[] for name in models.keys()}
train_mse = {name:[] for name in models.keys()}
orig_stability = {name:[] for name in models.keys()}
orig_standard_stability = {name:[] for name in models.keys()}
orig_mse = {name:[] for name in models.keys()}

kf = RepeatedKFold(n_splits= 5,n_repeats=10, random_state=SEED)
for train_index, test_index in kf.split(X):
    X_12, y_12 = X[train_index],y[train_index]
    ClaimNb_12 = ClaimNb[train_index]
    Exposure_12  = Exposure[train_index]
    X_test,y_test = X[test_index],y[test_index]
    ClaimNb_test = ClaimNb[test_index]
    Exposure_test  = Exposure[test_index]
    X1,X2,y1,y2,ClaimNb_1,ClaimNb_2,Exposure_1,Exposure_2 =  train_test_split(X_12, y_12,ClaimNb_12,Exposure_12, test_size=0.5, random_state=SEED)
   
    # initial model 
    criterion = "poisson"
    models = {  
                "baseline": BaseLineTree(criterion = criterion, adaptive_complexity=True),
                "NU": NaiveUpdate(criterion = criterion, adaptive_complexity=True),
            "TR":TreeReevaluation(criterion = criterion, adaptive_complexity=True, delta=0.1),
            "SR":StabilityRegularization(criterion = criterion, adaptive_complexity=True,lmbda=0.75),
            #"BT": BootstrapUpdate(criterion = criterion, adaptive_complexity=True),
            "ABU":AbuTreeI(criterion = criterion, adaptive_complexity=True)

            }
    for name, model in models.items():
        model.fit(X1,y1)
        
        pred1 = model.predict(X_test)
        pred1_train = model.predict(X_12)
        pred1_orig= model.predict(X1)
        #print("before")
        model.update(X_12,y_12)
        #print("after")
        pred2 = model.predict(X_test)
        pred2_orig= model.predict(X1)
        pred2_train =  model.predict(X_12)
        mean_poisson_deviance(ClaimNb,t.predict(X)*Exposure)
        orig_mse[name].append(mean_poisson_deviance(ClaimNb_1+0.00001,pred2_orig*Exposure_1+0.00001))
        orig_stability[name].append(S1(pred1_orig*Exposure_1,pred2_orig*Exposure_1))
        orig_standard_stability[name].append(S2(pred1_orig*Exposure_1,pred2_orig*Exposure_1))

        train_mse[name].append(mean_poisson_deviance(ClaimNb_12+0.00001,pred2_train*Exposure_12+0.00001))
        train_stability[name].append(S1(pred1_train*Exposure_12,pred2_train*Exposure_12))
        train_standard_stability[name].append(S2(pred1_train*Exposure_12,pred2_train*Exposure_12))
        mse[name].append(mean_poisson_deviance(ClaimNb_test+0.00001,pred2*Exposure_test+0.00001))
        stability[name].append(S1(pred1*Exposure_test,pred2*Exposure_test))
        standard_stability[name].append(S2(pred1*Exposure_test,pred2*Exposure_test))
    
for name in models.keys():
    print("="*80)
    print(f"{name}")
    orig_mse_scale = np.mean(orig_mse["baseline"]); orig_S1_scale = np.mean(orig_stability["baseline"]); orig_S2_scale = np.mean(orig_standard_stability["baseline"]);
    train_mse_scale = np.mean(train_mse["baseline"]); train_S1_scale = np.mean(train_stability["baseline"]); train_S2_scale = np.mean(train_standard_stability["baseline"]);
    mse_scale = np.mean(mse["baseline"]); S1_scale = np.mean(stability["baseline"]); S2_scale = np.mean(standard_stability["baseline"]);
    
    print(f"orig - mse: {np.mean(orig_mse[name]):.3f} ({np.mean(orig_mse[name])/orig_mse_scale:.2f}), stability: {np.mean(orig_stability[name]):.3f} ({np.mean(orig_stability[name])/orig_S1_scale:.2f}), standard stability: {np.mean(orig_standard_stability[name]):.3f} ({np.mean(orig_standard_stability[name])/orig_S2_scale:.2f})")
    print(f"train - mse: {np.mean(train_mse[name]):.3f} ({np.mean(train_mse[name])/train_mse_scale:.2f}), stability: {np.mean(train_stability[name]):.3f} ({np.mean(train_stability[name])/train_S1_scale:.2f}), standard stability: {np.mean(train_standard_stability[name]):.3f} ({np.mean(train_standard_stability[name])/train_S2_scale:.2f})")
    print(f"test - mse: {np.mean(mse[name]):.3f} ({np.mean(mse[name])/mse_scale:.2f}), stability: {np.mean(stability[name]):.3f} ({np.mean(stability[name])/S1_scale:.2f}), standard stability: {np.mean(standard_stability[name]):.3f} ({np.mean(standard_stability[name])/S2_scale:.2f})")
    print("="*80)
    mse_all[name] += [score/mse_scale for score in mse[name]]
    stability_all[name] += [score/S1_scale for score in stability[name]]
    standard_stability_all[name] += [score/S2_scale for score in standard_stability[name]]
print()

baseline
orig - mse: 0.279 (1.00), stability: 0.002 (1.00), standard stability: 0.000 (1.00)
train - mse: 0.279 (1.00), stability: 0.002 (1.00), standard stability: 0.000 (1.00)
test - mse: 0.279 (1.00), stability: 0.002 (1.00), standard stability: 0.000 (1.00)
NU
orig - mse: 0.279 (1.00), stability: 0.001 (0.36), standard stability: 0.000 (0.14)
train - mse: 0.279 (1.00), stability: 0.001 (0.36), standard stability: 0.000 (0.14)
test - mse: 0.280 (1.00), stability: 0.001 (0.36), standard stability: 0.000 (0.14)
TR
orig - mse: 0.279 (1.00), stability: 0.001 (0.36), standard stability: 0.000 (0.14)
train - mse: 0.279 (1.00), stability: 0.001 (0.36), standard stability: 0.000 (0.14)
test - mse: 0.280 (1.00), stability: 0.001 (0.36), standard stability: 0.000 (0.14)
SR
orig - mse: 0.279 (1.00), stability: 0.001 (0.53), standard stability: 0.000 (0.28)
train - mse: 0.279 (1.00), stability: 0.001 (0.53), standard stability: 0.000 (0.28)
test - mse: 0.279 (1.00), stability: 0.001 (0.52), sta

In [22]:
x = np.random.uniform(size=(10000,2),low=0,high=10)
y = np.random.poisson(size = 10000, lam=x[:,0]+x[:,1] + np.clip(np.log(x[:,0]+x[:,1]),a_min=0, a_max=np.inf))

In [23]:
t = BaseLineTree(criterion="poisson",adaptive_complexity=True).fit(x,y)
mean_poisson_deviance(y,t.predict(x))

1.0213581223983699